In [ ]:
from pathlib import Path
from aicsimageio import AICSImage
import xml.etree.ElementTree as ET
import xmltodict
import pandas as pd

In [ ]:
input_dirpath = input('Please enter the path for the folder containing images')

In [ ]:
df = pd.DataFrame()

input_dirpath = Path(input_dirpath)
imgpaths = [path for path in input_dirpath.glob('*.czi')]

for i, imgpath in enumerate(imgpaths):
    print(f'Checking img ({i+1}/{len(imgpaths)})')
    
    ch_info_d = {}
    ch_info_d['imgname'] = Path(imgpath).name
    
    # get CZI metadata
    img = AICSImage(imgpath)
    metadata = img.metadata
    xmlstr = ET.tostring(metadata)
    metadatadict_czi = xmltodict.parse(xmlstr)
    
    channels_info = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['Channels']['Channel']
    for ch in range(len(channels_info)):
        ch_info_d[f'Ch {ch} name'] = [channels_info[ch]['@Name']]
        ch_info_d[f'Ch {ch} intensity'] = [channels_info[ch]['LightSourcesSettings']['LightSourceSettings']['Intensity']]
        ch_info_d[f'Ch{ch} exp (ms)'] = [int(channels_info[ch]['ExposureTime'])/1000000]
    
    df = pd.concat([df, pd.DataFrame(ch_info_d)], ignore_index=True)

df_savepath = input_dirpath/'imaging_settings_bych.csv'
df.to_csv(df_savepath)
print(f'Imaging settings saved to {df_savepath}')